<a href="https://www.kaggle.com/code/ayushkhaire/crypto-dataset-daily-updates-top-1000?scriptVersionId=193138666" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import requests as rq
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os
from tqdm import tqdm
from kaggle_secrets import UserSecretsClient
from datetime import datetime
import json
import subprocess
import logging
import time

# Yeasterday data :)

In [2]:
df = pd.read_csv('/kaggle/input/top-1000-cryptos-historical/1000_cryptos.csv')
df

,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,0,2015-01-01,320.434998,320.434998,314.002991,314.248993,314.248993,8.036550e+06,BTC-USD
1,1,2015-01-02,314.079010,315.838989,313.565002,315.032013,315.032013,7.860650e+06,BTC-USD
2,2,2015-01-03,314.846008,315.149994,281.082001,281.082001,281.082001,3.305440e+07,BTC-USD
3,3,2015-01-04,281.145996,287.230011,257.612000,264.195007,264.195007,5.562910e+07,BTC-USD
4,4,2015-01-05,265.084015,278.341003,265.084015,274.473999,274.473999,4.396280e+07,BTC-USD
...,...,...,...,...,...,...,...,...,...
50203,50203,2024-08-13,2724.303467,2737.990479,2613.801270,2703.671875,2703.671875,1.638305e+10,LTC-USD
50204,50204,2024-08-14,2703.586914,2775.281006,2636.705322,2662.914795,2662.914795,1.582537e+10,LTC-USD
50205,50205,2024-08-15,2662.885498,2675.314453,2518.670654,2570.087646,2570.087646,1.634580e+10,LTC-USD
50206,50206,2024-08-16,2570.085205,2629.383545,2553.071533,2593.186523,2593.186523,1.327556e+10,LTC-USD


# Define headers

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

# Extract symbols

In [4]:
 link = 'https://finance.yahoo.com/crypto/?offset=0&count=100'

# Get symbols and hit urls

In [5]:
resp_main_list = []
i = 0
for page in tqdm(range(40)):
    url = f'https://finance.yahoo.com/crypto/?offset={i*100}&count=25'
    r = rq.get(url,headers = headers)
    resp_main_list.append(r)
    i = i+1
#     print(url)

100%|██████████| 40/40 [00:43<00:00,  1.10s/it]


In [6]:
symbol_list = []

for r in resp_main_list:
    soup = BeautifulSoup(r.text,'html.parser')
    symbs= soup.find_all('span',{'class':'symbol yf-ravs5v'})
    for s in symbs:
        symbol_list.append(s.text)
        
print(len(symbol_list))

1000


In [7]:
unique_symbol_list = []

for sy in symbol_list:
    if sy not in unique_symbol_list:
        unique_symbol_list.append(sy)

unique_symbol_list = list(set(unique_symbol_list))
print(len(unique_symbol_list))

25


# Now , get the link for today downloads from a tesla stock 

### This we willl use to use "code" by yahoo finance to figure out dates 

In [8]:
'''

link1 = https://query1.finance.yahoo.com/v7/finance/download/TSLA?period1=1684241704&period2=1715864104&interval=1d&events=history&includeAdjustedClose=true

link2 = https://query1.finance.yahoo.com/v7/finance/download/AMD?period1=1684241769&period2=1715864169&interval=1d&events=history&includeAdjustedClose=true


here the period values are exactly same.
'''

# After research , i got they are unix timestamps in period . lets try it out !

import time

# Get current Unix timestamp
current_timestamp = int(time.time())

date_str = "2015-01-01"
date_obj = datetime.strptime(date_str, "%Y-%m-%d")
period1 = int(time.mktime(date_obj.timetuple()))


period2 = current_timestamp  

period1, period2 = period2, period1

# Print the timestamps in the desired format
print(f"period1={period1}&period2={period2}")


period1=1724040220&period2=1420070400


# create stocks directory

In [9]:
os.mkdir('crypto_data')

# Download all files

In [10]:
for stock in tqdm(symbol_list[:1000]):
    url = f"https://query1.finance.yahoo.com/v7/finance/download/{stock}?period1={period2}&period2={period1}&interval=1d&events=history&includeAdjustedClose=true"
#     print(url)
    r = rq.get(url, headers=headers)
    if r.status_code == 200:
        with open(f"crypto_data/{stock}.csv", "wb") as file:
            file.write(r.content)
#         print(f"File downloaded successfully : {stock}.csv")
#     else:
#         print(f"Failed to download file : {stock}.csv")
#     time.sleep(3)


# https://query1.finance.yahoo.com/v7/finance/download/AMD?period1=1684244457&period2=1715866857&interval=1d&events=history&includeAdjustedClose=true

100%|██████████| 1000/1000 [07:40<00:00,  2.17it/s]


# Read a sample

In [11]:
first_file = os.listdir('/kaggle/working/crypto_data')[0]
print(first_file)
df = pd.read_csv(f'/kaggle/working/crypto_data/{first_file}')
df

WTRX-USD.csv


,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-03-05,0.060484,0.060734,0.060390,0.060396,0.060396,1242664.0
1,2022-03-06,0.060396,0.060712,0.058954,0.058955,0.058955,1147624.0
2,2022-03-07,0.058956,0.060431,0.058014,0.058566,0.058566,1896907.0
3,2022-03-08,0.058564,0.060910,0.058564,0.060757,0.060757,1658131.0
4,2022-03-09,0.060757,0.062442,0.060578,0.060668,0.060668,2175214.0
...,...,...,...,...,...,...,...
894,2024-08-15,0.130224,0.131657,0.130126,0.130179,0.130179,1600481.0
895,2024-08-16,0.130179,0.133722,0.129373,0.133722,0.133722,4374564.0
896,2024-08-17,0.133722,0.136045,0.133666,0.135050,0.135050,8776062.0
897,2024-08-18,NaN,NaN,NaN,NaN,NaN,NaN


# Work on dataset

In [12]:
csvlist = os.listdir('/kaggle/working/crypto_data')

In [13]:
megadf = pd.DataFrame()

In [14]:
for sym, file in zip(symbol_list, csvlist):
    df = pd.read_csv(f'/kaggle/working/crypto_data/{file}')
    df['Symbol'] = sym
    megadf = pd.concat([megadf, df], ignore_index=True)

In [15]:
megadf

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,2022-03-05,0.060484,0.060734,0.060390,0.060396,0.060396,1242664.0,BTC-USD
1,2022-03-06,0.060396,0.060712,0.058954,0.058955,0.058955,1147624.0,BTC-USD
2,2022-03-07,0.058956,0.060431,0.058014,0.058566,0.058566,1896907.0,BTC-USD
3,2022-03-08,0.058564,0.060910,0.058564,0.060757,0.060757,1658131.0,BTC-USD
4,2022-03-09,0.060757,0.062442,0.060578,0.060668,0.060668,2175214.0,BTC-USD
...,...,...,...,...,...,...,...,...
50253,2024-08-15,6.908892,6.909690,6.456883,6.616733,6.616733,450312244.0,LTC-USD
50254,2024-08-16,6.616733,6.865416,6.409167,6.545818,6.545818,500857409.0,LTC-USD
50255,2024-08-17,6.545818,6.572308,6.405538,6.440839,6.440839,185786009.0,LTC-USD
50256,2024-08-18,NaN,NaN,NaN,NaN,NaN,NaN,LTC-USD


In [16]:
first_date = megadf['Date'][0]
last_date = megadf['Date'].iloc[-1]

In [17]:
first_date

'2022-03-05'

In [18]:
megadf.dtypes

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume       float64
Symbol        object
dtype: object

In [19]:
megadf['Date'] = pd.to_datetime(megadf['Date'], format="%Y-%m-%d")

In [20]:
megadf.dtypes

Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume              float64
Symbol               object
dtype: object

In [21]:
os.mkdir('dataset')

In [22]:
megadf.to_csv(f'dataset/1000_cryptos.csv')

# final dataset

In [23]:
df = pd.read_csv(f'/kaggle/working/dataset/1000_cryptos.csv')

In [24]:
df

,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,0,2022-03-05,0.060484,0.060734,0.060390,0.060396,0.060396,1242664.0,BTC-USD
1,1,2022-03-06,0.060396,0.060712,0.058954,0.058955,0.058955,1147624.0,BTC-USD
2,2,2022-03-07,0.058956,0.060431,0.058014,0.058566,0.058566,1896907.0,BTC-USD
3,3,2022-03-08,0.058564,0.060910,0.058564,0.060757,0.060757,1658131.0,BTC-USD
4,4,2022-03-09,0.060757,0.062442,0.060578,0.060668,0.060668,2175214.0,BTC-USD
...,...,...,...,...,...,...,...,...,...
50253,50253,2024-08-15,6.908892,6.909690,6.456883,6.616733,6.616733,450312244.0,LTC-USD
50254,50254,2024-08-16,6.616733,6.865416,6.409167,6.545818,6.545818,500857409.0,LTC-USD
50255,50255,2024-08-17,6.545818,6.572308,6.405538,6.440839,6.440839,185786009.0,LTC-USD
50256,50256,2024-08-18,NaN,NaN,NaN,NaN,NaN,NaN,LTC-USD


# Upload Dataset

# User secrets

In [25]:
user_secrets = UserSecretsClient()
kaggle_apikey = user_secrets.get_secret("kaggle_apikey")
kaggle_username = user_secrets.get_secret("kaggle_username")

os.environ['KAGGLE_USERNAME'] = kaggle_username
os.environ['KAGGLE_KEY'] = kaggle_apikey

# Metadata file

In [26]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print('Starting upload >>>>>>') 
print('Creating metadata file >>>>')
data = {
    "id": "ayushkhaire/top-1000-cryptos-historical"
}
metadata_file_location = '/kaggle/working/dataset/dataset-metadata.json' 
with open(metadata_file_location, 'w', encoding='utf-8') as metadata_file:
    json.dump(data, metadata_file)
print('Metadata file created')

Starting upload >>>>>>
Creating metadata file >>>>
Metadata file created


# upload

In [27]:
retries = 0
while retries < 5:
    try:
        command = "kaggle datasets version -p '/kaggle/working/dataset' -m 'Update' -r zip"
        subprocess.run(command, shell=True, check=True)
        logger.info("Upload complete")
        break
    except Exception as error:
        logger.error(f"Error from Kaggle: {error}")
        time.sleep(5)
        retries += 1

Starting upload for file 1000_cryptos.csv


100%|██████████| 4.16M/4.16M [00:02<00:00, 2.10MB/s]


Upload successful: 1000_cryptos.csv (4MB)
Dataset version is being created. Please check progress at https://www.kaggle.com/ayushkhaire/top-1000-cryptos-historical


# Thank you , and I will see you in next notebook :)